In [49]:
import librosa
import numpy as np
import os
import pickle

g=os.listdir("wav_files")

spectrals=[]
for vidfile in g:

    dr=librosa.get_duration(path=f"wav_files/{vidfile}")
    y,sr = librosa.load(f"wav_files/{vidfile}",sr=10000/dr)



    #25k  points of sampled time serie data
    #print(f"Sampling data: {y.shape}")
    print(f"Sample rate: {sr}")
    mfccs = librosa.feature.mfcc(y=y, sr=sr)

    #print(mfccs.shape)

    mfccs_mean = np.mean(mfccs, axis=0)[::2]
    mfccs_max = np.max(mfccs, axis=0)[::2]

    #print(mfccs_max.shape)
    #print(mfccs_mean.shape)


    zero_crossings = librosa.zero_crossings(y, pad=False)
    #print(np.array((sum(zero_crossings))))
    song_characteristics=np.concatenate((mfccs_mean,mfccs_max))
    song_characteristics=np.concatenate((song_characteristics,[sum(zero_crossings)]))
    spectrals.append(song_characteristics)
    #https://towardsdatascience.com/extract-features-of-music-75a3f9bc265dnp.array(sum(zero_crossings))
spectrals=np.array(spectrals)
print(spectrals.shape)


Sample rate: 43.64248707944872
Sample rate: 36.82778027193432
Sample rate: 44.03159251786425
Sample rate: 47.49272855094839
Sample rate: 43.256735887906785
Sample rate: 51.033822579122095
Sample rate: 31.421571756894792
Sample rate: 42.58940491495253
Sample rate: 35.67462413021869
Sample rate: 39.11288777634453
Sample rate: 33.60887018105197
Sample rate: 48.15764782759867
Sample rate: 37.61915290880503
Sample rate: 38.93607914212782
Sample rate: 42.54732883817427
Sample rate: 38.247252442273535
Sample rate: 40.4151710304054
Sample rate: 20.554794888316152
Sample rate: 41.1882232689365
Sample rate: 40.85687900138005
Sample rate: 33.956552860111934
Sample rate: 48.8680345874873
Sample rate: 42.35985707150267
Sample rate: 34.4421035268714
Sample rate: 34.74778622720671
Sample rate: 35.06465254030288
Sample rate: 51.49761125819524
Sample rate: 54.29451115733737
Sample rate: 36.17809664818549
Sample rate: 83.1912228543833
Sample rate: 50.13086315115558
Sample rate: 54.954092771480695
Sample

In [50]:
import sklearn
#Mean, max, zero
spectrals_n=sklearn.preprocessing.normalize(spectrals,axis=0)
print(spectrals_n.shape)


(33, 21)


In [51]:
from sklearn.decomposition import PCA

#For means
spectral_n_means=spectrals_n[:,:10]
print(spectral_n_means.shape)
pca = PCA(n_components=5)
 
pca_means=pca.fit_transform(spectral_n_means)
#print(pca.components_)
#print(pca.explained_variance_)

print(pca_means.shape)

#retain 5

(33, 10)
(33, 5)


In [52]:
#For maxes
spectral_n_maxes=spectrals_n[:,10:20]
#print(spectral_n_maxes.shape)
pca = PCA(n_components=5)
 
pca_maxes=pca.fit_transform(spectral_n_maxes)
#print(pca.components_)
#print(pca.explained_variance_)
print(pca_maxes.shape)

#retain 5

(33, 5)


In [53]:
reduced_spectral=np.concatenate((pca_maxes,pca_means),axis=1)
print(spectrals_n[:,20:].shape)
reduced_spectral=np.concatenate((reduced_spectral,spectrals_n[:,20:]),axis=1)
print(reduced_spectral.shape)

(33, 1)
(33, 11)


In [ ]:
distortions = []
inertias = []
mapping1 = {}
mapping2 = {}
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist
K = range(1, 10)

for k in K:
	# Building and fitting the model
	kmeanModel = KMeans(n_clusters=k).fit(reduced_spectral)
	kmeanModel.fit(reduced_spectral)

	distortions.append(sum(np.min(cdist(reduced_spectral, kmeanModel.cluster_centers_,
										'euclidean'), axis=1)) / reduced_spectral.shape[0])
	inertias.append(kmeanModel.inertia_)

	mapping1[k] = sum(np.min(cdist(reduced_spectral, kmeanModel.cluster_centers_,
								'euclidean'), axis=1)) / reduced_spectral.shape[0]
	mapping2[k] = kmeanModel.inertia_


In [ ]:
!python -m pip install matplotlib
import matplotlib.pyplot as plt
plt.plot(K, distortions, 'bx-')
plt.xlabel('Values of K')
plt.ylabel('Distortion')
plt.title('The Elbow Method using Distortion')
plt.show()

In [62]:

kmeanModel = KMeans(n_clusters=4).fit(reduced_spectral)
kmeanModel.fit(reduced_spectral)
print(kmeanModel.labels_)

/home/hakosaj/.pyenv/versions/3.11.3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/home/hakosaj/.pyenv/versions/3.11.3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


[2 0 1 2 1 0 0 0 1 1 2 2 0 3 1 2 1 1 3 1 2 1 2 0 2 0 1 3 0 0 3 3 0]
